# Sprint19 Unet

# 事前準備

https://github.com/zhixuhao/unet よりダウンロードしてきたプログラム一式が入ったディレクトリの中に、このSprint19.ipynbを入れる

# 問題1　学習と推定

まずは、main.pyが正常に動くことを確認しておく。

その後、https://www.kaggle.com/c/tgs-salt-identification-challenge/data データを使って、学習を行っていきます。

## ①サンプルデータセットで動かしてみる

In [ ]:
!python main.py

## ②データ準備

unetのcloneしてきたフォルダの中に、このsprint19.ipynbファイルが入っているかと思います。

この状態で、https://www.kaggle.com/c/tgs-salt-identification-challenge/data からダウンロードしてきたデータを、unetフォルダの中に置きましょう。

最終的に、下記のようなフォルダ・ファイル構成になっているものとします。

unet/

    ・・・・
    Sprint19.ipynb
    competition_data/
        train/
        test/
        depths.csv
    ・・・・

## ③ライブラリのimport

この③以降は、unetリポジトリの内部のコードを抜粋してきてあります。

In [ ]:
import glob
import cv2
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 9)

## ④データ読み込み

In [ ]:
# 画像パス一覧
image_path_list = glob.glob("competition_data/train/images/*")

# データ格納用リスト
X_train=[]
y_train=[]

# 画像パスでループ
for image_path in image_path_list[:100]:
    # 画像
    image = cv2.resize(cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)/255, (256, 256))
    X_train.append(image)
    # セグメンテーション領域
    mask = cv2.resize(cv2.imread(image_path.replace("images","masks"), cv2.IMREAD_GRAYSCALE)/255, (256, 256))
    y_train.append(mask)


# numpy配列化
X_train=np.array(X_train)
y_train=np.array(y_train)
print(X_train.shape, y_train.shape)

# 学習用に形整える
X_train = np.expand_dims(X_train, axis=-1)
y_train = np.expand_dims(y_train, axis=-1)
print(X_train.shape, y_train.shape)

## ⑤モデル定義

In [ ]:
def unet(pretrained_weights = None,input_size = (256,256,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(input = inputs, output = conv10)

    model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    if(pretrained_weights):
        model.load_weights(pretrained_weights)

    return model

model = unet()

## ⑥学習

In [ ]:
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit(X_train,y_train,batch_size=3,epochs=100,callbacks=[model_checkpoint])

## ⑦推定

In [ ]:
y_pred = model.predict(np.array([X_train[0,:,:]]))

In [ ]:
fig, ax = plt.subplots(1, 2)
ax[0].imshow(y_pred[0,:,:,0], cmap='gray')
ax[1].imshow(y_train[0], cmap='gray')

# 問題2　コードリーディング

https://github.com/zhixuhao/unet

- 逆畳み込みにより、畳み込み前の大きさに近づけていっている（upsampling）
- 逆畳み込み時は、同階層の者をコピペしてきている(copy)